In [68]:
import numpy as np
import torch
from skimage.io import imsave
from torch.autograd import Variable
from machinedesign.viz import grid_of_images_default

from scipy.spatial.distance import cdist
from lapjv import lapjv

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def grid_embedding(h):
    assert int(np.sqrt(h.shape[0])) ** 2 == h.shape[0], 'Nb of examples must be a square number'
    size = np.sqrt(h.shape[0])
    grid = np.dstack(np.meshgrid(np.linspace(0, 1, size), np.linspace(0, 1, size))).reshape(-1, 2)
    cost_matrix = cdist(grid, h, "sqeuclidean").astype('float32')
    cost_matrix = cost_matrix * (100000 / cost_matrix.max())
    _, rows, cols = lapjv(cost_matrix)
    return rows


In [60]:
folder = 'results/celeba'
gen = torch.load('{}/gen.th'.format(folder), map_location=lambda storage, loc: storage)
gen.train()
nz = gen.nz

In [70]:
nz = gen.nz
nb_blocks = nz // gen.nz_per_block
nb_factors_per_block = gen.nb_factors // nb_blocks
nb = 20*20
np.random.seed(42)
for i in range(0, nz, gen.nz_per_block):
    Z = np.random.uniform(-1, 1, size=(1, nz)) * np.ones((nb, 1))
    Z[:, i:i + nb_factors_per_block] = np.random.uniform(-1, 1, size=(nb, nb_factors_per_block))
    Z = Z.astype(np.float32)
    #p = PCA(n_components=2).fit_transform(Z)
    #r = grid_embedding(p)

    Z = torch.from_numpy(Z)
    Z = Z.view(Z.size(0), Z.size(1), 1, 1)
    Z = Variable(Z)
    x = gen(Z)
    x = x.data.numpy()
    
    p = TSNE(n_components=2).fit_transform(x.reshape((x.shape[0], -1)))
    p = grid_embedding(p)
    x = x[p]
    print(x.shape)
    x = (x + 1) / 2
    im = grid_of_images_default(x, normalize=False)
    print(i//gen.nz_per_block)
    imsave('latent{}.png'.format(i // gen.nz_per_block), im)

/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:16: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


(400, 3, 64, 64)
0


/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


(400, 3, 64, 64)
1
(400, 3, 64, 64)
2
(400, 3, 64, 64)
3
